In [1]:
import os
import sys
sys.path.append('./bert')
import lm_model
import tensorflow as tf
import tokenization
from optimization import create_optimizer
import model_transfer as mtransfer

In [2]:
config = lm_model.BASE_PARAMS
tokenizer = tokenization.FullTokenizer('./model/vocab.txt')
config['vocab_size'] = len(tokenizer.vocab)
config['max_length'] = 64
config['batch_size'] = 4
vocab_file = './model/vocab.txt'
train_file = './train.tfrecord'

In [3]:
# processor = lm_model.LMWikiDataProcessor()
# ds = lm_model.LMDataSet('./model/vocab.txt', config['max_length'])
# d = ds.get_ds('./train.tfrecord', config['batch_size'])

In [4]:
# train_iterator = d.make_one_shot_iterator()
# train_inputs = train_iterator.get_next()

In [5]:
# with tf.Session() as sess:
#     train_inputs_eval = sess.run(train_inputs)
#     print(train_inputs_eval)
#     print(tokenizer.convert_ids_to_tokens(train_inputs_eval['text'][0]))

In [6]:
def make_feed_dict(model, inputs):
    dicts = {
        model.inp : inputs['text'], 
        model.inp_len : inputs['text_len'] 
    }
    return dicts

In [7]:
# model = lm_model.LMModel(config, config['max_length'])
# loss = model.loss(True)
# train_op = create_optimizer(loss, 0.01, 100000, 10000, False)

In [8]:
# loop_num = 0
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     while loop_num < 200:
#         loop_num += 1
#         inputs = sess.run(train_inputs)
#         inputs_dict = make_feed_dict(model, inputs)
#         loss_val = sess.run([loss, train_op], inputs_dict)
#         print(loss_val)

In [9]:
def train(train_file, vocab_file, config, log_dir, pretrained=None):
    graph = tf.Graph()
    with graph.as_default():
        ds = lm_model.LMDataSet(vocab_file, config['max_length'])
        d = ds.get_ds(train_file, config['batch_size'])
        train_iterator = d.make_one_shot_iterator()
        train_inputs = train_iterator.get_next()
        model = lm_model.LMModel(config, config['max_length'])
        loss = model.loss(True)
        train_op = create_optimizer(loss, config['learning_rate'], config['train_steps'], config['learning_rate_warmup_steps'], False)
        partialSaver = None
        if pretrained:
            partialSaver = mtransfer.partial_transfer(pretrained)
        sv = tf.train.Supervisor(graph=graph, logdir=log_dir)
        with sv.managed_session(master='') as sess:
            train_steps = config['train_steps']
#             sess.run(tf.global_variables_initializer())
            if partialSaver:
                partialSaver.restore(sess, pretraied)
            for step in range(train_steps):
                if sv.should_stop():
                    break
                try:
                    inputs = sess.run(train_inputs)
                    feed_dicts = make_feed_dict(model, inputs)
                    loss_val, _ = sess.run([loss, train_op], feed_dict=feed_dicts)
                    if (step+1)%1:
                        print('====> [{}/{}]\tloss:{.3f}'.format(step, train_steps, loss_val))
                except:
                    sv.saver.save(sess, './final_model', global_step=(step+1))
            sess.run(tf.global_variables_initializer())
        

In [ ]:
train('./train.tfrecord', './model/vocab.txt', config, './log')

/Users/higgs/anaconda2/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from ./log/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ./log/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global_step/sec: 1.31669
INFO:tensorflow:global_step/sec: 1.42501
INFO:tensorflow:global_step/sec: 1.48336
INFO:tensorflow:global_step/sec: 1.49166
INFO:tensorflow:Saving checkpoint to path ./log/model.ckpt
INFO:tensorflow:global_step/sec: 1.49995
INFO:tensorflow:global_step/sec: 1.51672
INFO:tensorflow:global_step/sec: 1.50831


In [ ]:
# graph = tf.Graph()
# with graph.as_default():
#     ds = lm_model.LMDataSet(vocab_file, config['max_length'])
#     d = ds.get_ds(train_file, config['batch_size'])
#     train_iterator = d.make_one_shot_iterator()
#     train_inputs = train_iterator.get_next()
#     model = lm_model.LMModel(config, config['max_length'])
#     loss = model.loss(True)
#     train_op = create_optimizer(loss, config['learning_rate'], config['train_steps'], config['learning_rate_warmup_steps'], False)
#     partialSaver = None
#     sv = tf.train.Supervisor(graph=graph, logdir='./log')
#     with sv.managed_session(master='') as sess:
# #         sess.run(tf.global_variables_initializer())
#         inputs = sess.run(train_inputs)
#         print(inputs)

